# 微调 Open AI 模型

本笔记本基于 Open AI 当前提供的[微调](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst)文档中的指导。

微调通过使用与特定用例或场景相关的额外数据和上下文重新训练基础模型，从而提升其在您的应用中的性能。请注意，提示工程技术如_少量示例学习_和_检索增强生成_允许您使用相关数据增强默认提示以提高质量。然而，这些方法受限于目标基础模型的最大令牌窗口大小。

通过微调，我们实际上是用所需数据重新训练模型本身（允许我们使用比最大令牌窗口能容纳的更多示例）——并部署一个_定制_版本的模型，该模型在推理时不再需要提供示例。这不仅提升了我们提示设计的有效性（我们在使用令牌窗口时有更多灵活性），还可能降低成本（通过减少推理时需要发送给模型的令牌数量）。

微调有四个步骤：
1. 准备训练数据并上传。
1. 运行训练任务以获得微调模型。
1. 评估微调模型并迭代以提升质量。
1. 满意后部署微调模型进行推理。

请注意，并非所有基础模型都支持微调——[查看 OpenAI 文档](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst)获取最新信息。您也可以对先前微调过的模型进行微调。在本教程中，我们将使用 `gpt-35-turbo` 作为微调的目标基础模型。

---


### 第 1.1 步：准备您的数据集

让我们构建一个聊天机器人，通过用打油诗回答有关元素的问题，帮助您了解元素周期表。在_本_简单教程中，我们将仅创建一个数据集，用几个示例响应来训练模型，展示数据的预期格式。在实际应用中，您需要创建一个包含更多示例的数据集。如果存在适用于您的应用领域的开放数据集，您也可以使用它，并重新格式化以用于微调。

由于我们专注于 `gpt-35-turbo` 并寻求单轮响应（聊天完成），我们可以使用[此建议格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)创建示例，符合 OpenAI 聊天完成的要求。如果您期望多轮对话内容，则应使用[多轮示例格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，其中包含一个 `weight` 参数，用于指示哪些消息应在微调过程中使用（或不使用）。

在本教程中，我们将使用更简单的单轮格式。数据采用[jsonl 格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每行一个记录，每个记录表示为 JSON 格式的对象。下面的代码片段显示了 2 条记录作为示例——完整示例集（10 个示例）请参见 [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，这是我们用于微调教程的示例。**注意：**每条记录_必须_定义在单行中（不能像格式化的 JSON 文件那样跨行拆分）

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

在实际应用中，您需要更多的示例集以获得良好效果——权衡点在于响应质量与微调所需的时间/成本。我们使用较小的示例集，以便快速完成微调，演示整个过程。有关更复杂的微调教程，请参见[此 OpenAI Cookbook 示例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)。


---

### 第 1.2 步 上传您的数据集

使用文件 API 上传数据，[如此处所述](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file)。请注意，为了运行此代码，您必须先完成以下步骤：
 - 安装 `openai` Python 包（确保使用版本 >=0.28.0 以获得最新功能）
 - 将 `OPENAI_API_KEY` 环境变量设置为您的 OpenAI API 密钥
要了解更多信息，请参阅课程提供的[设置指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

现在，运行代码以从本地 JSONL 文件创建一个用于上传的文件。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### 第2.1步：使用SDK创建微调作业


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### 第 2.2 步：检查作业状态

以下是您可以使用 `client.fine_tuning.jobs` API 执行的一些操作：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近的 n 个微调作业
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 获取特定微调作业的详细信息
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消微调作业
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出该作业的最多 n 条事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

该过程的第一步是_验证训练文件_，以确保数据格式正确。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### 第2.3步：跟踪事件以监控进度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 第2.4步：在OpenAI仪表板中查看状态


您还可以通过访问 OpenAI 网站并浏览平台的 _微调_ 部分来查看状态。这将显示当前作业的状态，并让您跟踪之前作业执行的历史记录。在此截图中，您可以看到之前的执行失败了，第二次运行成功了。作为背景说明，这发生在第一次运行时使用了格式错误的 JSON 文件记录——修正后，第二次运行成功完成，并使模型可供使用。

![Fine-tuning job status](../../../../../translated_images/zh-CN/fine-tuned-model-status.563271727bf7bfba.webp)


您还可以通过在可视化仪表板中向下滚动来查看状态消息和指标，如下所示：

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/zh-CN/fine-tuned-messages-panel.4ed0c2da5ea1313b.webp) |  ![Metrics](../../../../../translated_images/zh-CN/fine-tuned-metrics-panel.700d7e4995a65229.webp)|


---

### 第3.1步：在代码中检索ID并测试微调模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### 第3.2步：在Playground中加载并测试微调模型

您现在可以通过两种方式测试微调模型。首先，您可以访问Playground，并使用“Models”下拉菜单从列出的选项中选择您新微调的模型。另一种方式是使用微调面板中显示的“Playground”选项（见上方截图），它会启动以下的_对比_视图，显示基础模型和微调模型版本并排，便于快速评估。

![Fine-tuning job status](../../../../../translated_images/zh-CN/fine-tuned-playground-compare.56e06f0ad8922016.webp)

只需填写您训练数据中使用的系统上下文，并提供您的测试问题。您会注意到两边都会更新相同的上下文和问题。运行比较，您将看到它们输出的差异。_注意微调模型如何以您示例中提供的格式呈现响应，而基础模型则仅遵循系统提示_。

![Fine-tuning job status](../../../../../translated_images/zh-CN/fine-tuned-playground-launch.5a26495c983c6350.webp)

您会注意到比较还提供了每个模型的令牌计数和推理所用时间。**这个具体示例是一个简单示范，旨在展示流程，而不是真实世界的数据集或场景**。您可能会注意到两个样本显示的令牌数相同（系统上下文和用户提示相同），但微调模型推理时间更长（自定义模型）。

在真实场景中，您不会使用像这样的玩具示例，而是针对真实数据进行微调（例如，用于客户服务的产品目录），此时响应质量会更加明显。在_那个_背景下，使用基础模型获得等效响应质量将需要更多的自定义提示工程，这会增加令牌使用量，并可能增加相关的推理处理时间。_要尝试这个，请查看OpenAI Cookbook中的微调示例开始。_

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文件由人工智能翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译。尽管我们力求准确，但请注意自动翻译可能存在错误或不准确之处。原始文件的母语版本应被视为权威来源。对于重要信息，建议使用专业人工翻译。我们不对因使用本翻译而产生的任何误解或误释承担责任。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
